In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
#The following dataframe has been manually filtered using Wikipedia

df1 = pd.read_csv('D_ICD_DIAGNOSES_HEART_FILTERED.csv')
df1['ICD9_CODE'] = df1['ICD9_CODE'].astype(str)

In [ ]:
# Find total heart patients admitted in the hospital

df2 = pd.read_csv('DIAGNOSES_ICD.csv')
df2['ICD9_CODE'] = df2['ICD9_CODE'].astype(str)
res_heart_patients = pd.merge(df1, df2, on='ICD9_CODE')

res_heart_patients.SUBJECT_ID.nunique()

In [ ]:
# Save the dataframe

res_heart_patients.to_csv('HEART_PATIENTS_FILTERED.csv', index=False)

In [ ]:
#Load the Admission file which contains the detail of all the patient like Admission, Discharge Time etc.

df_ADMISSIONS = pd.read_csv('ADMISSIONS.csv')
df_ADMISSIONS = df_ADMISSIONS[df_ADMISSIONS.SUBJECT_ID.isin(list_patients)]
df_ADMISSIONS.columns = df_ADMISSIONS.columns.str.replace('HOSPITAL_EXPIRE_FLAG', 'DEATH_IN_HOSPITAL')
df_ADMISSIONS = df_ADMISSIONS.drop(columns=['ROW_ID', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'ETHNICITY','EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS'])
df_ADMISSIONS['ADMITTIME'] = pd.to_datetime(df_ADMISSIONS.ADMITTIME, format='%Y-%m-%d %H:%M:%S')
df_ADMISSIONS['DISCHTIME'] = pd.to_datetime(df_ADMISSIONS.DISCHTIME, format='%Y-%m-%d %H:%M:%S')
df_ADMISSIONS['DEATHTIME'] = pd.to_datetime(df_ADMISSIONS.DEATHTIME, format='%Y-%m-%d %H:%M:%S')
df_ADMISSIONS['DISEASE_DURATION'] = (df_ADMISSIONS.DISCHTIME-df_ADMISSIONS.ADMITTIME)/np.timedelta64(1, 'D')

In [ ]:
# Read the Patients file which contains factors like Gender, Expire Flag etc of the Patient

df_PATIENTS = pd.read_csv('PATIENTS.csv')

df_ADMISSIONS = pd.merge(df_ADMISSIONS,df_PATIENTS[['SUBJECT_ID','GENDER']],on='SUBJECT_ID', how='left')
df_ADMISSIONS.head()

df_ADMISSIONS = pd.merge(df_ADMISSIONS,df_PATIENTS[['SUBJECT_ID','DOB']],on='SUBJECT_ID', how='left')

df_ADMISSIONS = pd.merge(df_ADMISSIONS,df_PATIENTS[['SUBJECT_ID','DOD_SSN']],on='SUBJECT_ID', how='left')

df_ADMISSIONS['ADMITTIME'] = pd.to_datetime(df_ADMISSIONS['ADMITTIME']).dt.date
df_ADMISSIONS['DOB'] = pd.to_datetime(df_ADMISSIONS['DOB']).dt.date

df_ADMISSIONS['AGE'] = df_ADMISSIONS.apply(lambda e: (e['ADMITTIME'] - e['DOB']).days/365, axis=1)

for i in range(len(df_ADMISSIONS['AGE'])):
    if df_ADMISSIONS['AGE'][i] >= 300:
        df_ADMISSIONS['AGE'][i] = df_ADMISSIONS['AGE'][i] - 210

In [ ]:
df_ADMISSIONS.to_csv('ADMISSIONS_FILTERED.csv', index=False) 